# GET AVERAGE/MEDIAN PSNR SCORE & SIMILARITY SCORE ON VALIDATION DATASET FOR ANY MODEL

In [ ]:
import numpy as np
from math import log10
from skimage.metrics import structural_similarity as ssim
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
from database2 import DehazingDataset
import matplotlib.pyplot as plt
from skimage.transform import resize


# FOR PSNR/SSIM
import numpy as np
from math import log10
from skimage.metrics import structural_similarity as ssim
from torchvision.transforms import Lambda

# SIMPLE MODELS

In [ ]:
# Generator
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# Discriminator
# Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2),
            nn.Conv2d(256, 1, 4, stride=2, padding=1)
        )

    def forward(self, x):
        x = self.model(x)
        # Perform global average pooling
        x = torch.mean(x, dim=(2, 3))
        return torch.sigmoid(x)



# REFERENCED REPO MODELS

In [ ]:
 class CNNBlock(nn.Module):
    def __init__(self,in_channels, out_channels, stride = 2):
        super(CNNBlock,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,4,stride,padding_mode='reflect',bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self,x):
        return self.conv(x)


class Discriminator(nn.Module):
    def __init__(self,in_channels = 3, features = [64,128,256,512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2,features[0],kernel_size=4,stride=2,padding=1,padding_mode='reflect'),
            nn.LeakyReLU(0.2)
        ) # according to paper 64 channel doesn't contain BatchNorm2d
        layers = []
        in_channels = features[0]
        for feature in features[1:]:
            layers.append(CNNBlock(in_channels,feature,stride=1 if feature==features[-1] else 2 ))
            in_channels = feature
        
        layers.append(
            nn.Conv2d(in_channels,1,kernel_size=4,stride=1,padding=1,padding_mode='reflect')
        )
        self.model = nn.Sequential(*layers)
    
    def forward(self,x,y):
        x = torch.cat([x,y],dim=1)
        x = self.initial(x)
        x = self.model(x)
        return x

def test():
    x = torch.randn((1, 3, 256, 256))
    y = torch.randn((1, 3, 256, 256))
    model = Discriminator(in_channels=3)
    preds = model(x, y)
    print(model)
    print(preds.shape)

In [ ]:

class Block(nn.Module):
    def __init__(self,in_channels, out_channels, down = True, act = 'relu', use_dropout = False):
        super(Block,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size=4,stride=2,padding=1,bias=False,padding_mode='reflect')
            if down
            else
            nn.ConvTranspose2d(in_channels,out_channels,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act=='relu' else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
        self.down = down
    def forward(self,x):
        x = self.conv(x)
        return self.dropout(x) if self.use_dropout else x

class Generator(nn.Module):
    def __init__(self,in_channels=3,features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels,features,4,2,1,padding_mode='reflect'),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features,features*2,down=True,act='leaky',use_dropout=False)
        self.down2 = Block(features*2,features*4,down=True,act='leaky',use_dropout=False)
        self.down3 = Block(features*4,features*8,down=True,act='leaky',use_dropout=False)
        self.down4 = Block(features*8,features*8,down=True,act='leaky',use_dropout=False)
        self.down5 = Block(features*8,features*8,down=True,act='leaky',use_dropout=False)
        self.down6 = Block(features*8,features*8,down=True,act='leaky',use_dropout=False)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8,features*8,4,2,1,padding_mode='reflect'),
            nn.ReLU()
        )

        self.up1 = Block(features*8,features*8,down=False,act='relu',use_dropout=True)
        self.up2 = Block(features*8*2,features*8,down=False,act='relu',use_dropout=True)
        self.up3 = Block(features*8*2,features*8,down=False,act='relu',use_dropout=True)
        self.up4 = Block(features*8*2,features*8,down=False,act='relu',use_dropout=False)
        self.up5 = Block(features*8*2,features*4,down=False,act='relu',use_dropout=False)
        self.up6 = Block(features*4*2,features*2,down=False,act='relu',use_dropout=False)
        self.up7 = Block(features*2*2,features,down=False,act='relu',use_dropout=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2,in_channels,kernel_size=4,stride=2,padding=1),
            nn.Tanh(),
        )
    
    def forward(self,x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        d6 = self.down5(d5)
        d7 = self.down6(d6)
        bottleneck = self.bottleneck(d7)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1,d7],dim=1))
        up3 = self.up3(torch.cat([up2,d6],dim=1))
        up4 = self.up4(torch.cat([up3,d5],dim=1))
        up5 = self.up5(torch.cat([up4,d4],dim=1))
        up6 = self.up6(torch.cat([up5,d3],dim=1))
        up7 = self.up7(torch.cat([up6,d2],dim=1))
        return self.final_up(torch.cat([up7,d1],dim=1))

def test():
    x = torch.randn((1, 3, 256, 256))
    model = Generator(in_channels=3, features=64)
    preds = model(x)
    print(preds.shape)




# SIMPLE GENERATOR

In [ ]:


class Block(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act='relu'):
        super(Block, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False, padding_mode='reflect') if down
            else nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == 'relu' else nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.conv(x)

class SimpleGenerator(nn.Module):
    def __init__(self, in_channels=3, features=64):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, features, 4, 2, 1, padding_mode='reflect'),
            nn.LeakyReLU(0.2),
        )
        self.down1 = Block(features, features*2, down=True)
        self.down2 = Block(features*2, features*4, down=True)
        self.down3 = Block(features*4, features*8, down=True)
        self.down4 = Block(features*8, features*8, down=True)
        self.bottleneck = nn.Sequential(
            nn.Conv2d(features*8, features*8, 4, 2, 1, padding_mode='reflect'),
            nn.ReLU()
        )

        self.up1 = Block(features*8, features*8, down=False)
        self.up2 = Block(features*8*2, features*8, down=False)
        self.up3 = Block(features*8*2, features*4, down=False)
        self.up4 = Block(features*4*2, features*2, down=False)
        self.up5 = Block(features*2*2, features, down=False)
        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(features*2, in_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )
    
    def forward(self, x):
        d1 = self.initial_down(x)
        d2 = self.down1(d1)
        d3 = self.down2(d2)
        d4 = self.down3(d3)
        d5 = self.down4(d4)
        bottleneck = self.bottleneck(d5)
        up1 = self.up1(bottleneck)
        up2 = self.up2(torch.cat([up1, d5], dim=1))
        up3 = self.up3(torch.cat([up2, d4], dim=1))
        up4 = self.up4(torch.cat([up3, d3], dim=1))
        up5 = self.up5(torch.cat([up4, d2], dim=1))
        return self.final_up(torch.cat([up5, d1], dim=1))



# Baseline Generator

In [ ]:
class BasicGenerator(nn.Module):
    def __init__(self):
        super(BasicGenerator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

# MODEL - 4

In [ ]:
def down_conv(in_channels, out_channels, kernel_size, stride, padding):
  conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.LeakyReLU(0.2, inplace=True)
  )
  return conv

def up_conv(in_channels, out_channels, kernel_size, stride, padding):
  conv = nn.Sequential(
      nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True)
  )
  return conv

class GeneratorModel4(nn.Module):
    def __init__(self):
        super(GeneratorModel4, self).__init__()
        # Encoding layers
        self.down_conv_1 = down_conv(3,64,5,1,2)
        self.down_conv_2 = down_conv(64,128,4,2,1)
        self.down_conv_3 = down_conv(128,256,4,2,1)
        self.down_conv_4 = down_conv(256,512,4,2,1)

        self.up_trans_3 = up_conv(256, 128, 4, 2, 1)
        self.up_conv_3 = down_conv(256, 128, 3, 1, 1)
        self.up_trans_4 = up_conv(128, 64, 4, 2, 1)
        self.up_conv_4 = down_conv(128, 64, 3, 1, 1)

        self.out = nn.Conv2d(64, 3, kernel_size=1, stride=1, padding=0)

    def forward(self, image):
      #encoder
                                   # image = [32, 3, 256, 256]
      x1 = self.down_conv_1(image) # x1 = [32, 64, 256, 256]
      x2 = self.down_conv_2(x1)    # x2 = [32, 128, 128, 128]
      x3 = self.down_conv_3(x2)    # x3 = [32, 256, 64, 64]

      #decoder
      y = self.up_trans_3(x3)                   # y3 = [32, 128, 128, 128]
      y = self.up_conv_3(torch.cat([y,x2],1))  # y3 = [32, 128, 128, 128]
      y = self.up_trans_4(y)                   # y4 = [32, 64, 256, 256]
      y = self.up_conv_4(torch.cat([y,x1],1))  # y4 = [32, 64, 256, 256]
      y = self.out(y)

      return y

# MODEL - 5; HEAVY RESNET

In [ ]:
def down_conv(in_channels, out_channels, kernel_size, stride, padding):
  conv = nn.Sequential(
      nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.LeakyReLU(0.2, inplace=True)
  )
  return conv

def up_conv(in_channels, out_channels, kernel_size, stride, padding):
  conv = nn.Sequential(
      nn.ConvTranspose2d(in_channels, out_channels, kernel_size=kernel_size, stride=stride, padding=padding),
      nn.BatchNorm2d(out_channels),
      nn.ReLU(inplace=True)
  )
  return conv

class GeneratorModel5(nn.Module):
    def __init__(self):
        super(GeneratorModel5, self).__init__()
        # Encoding layers
        self.down_conv_1 = down_conv(3,64,5,1,2)
        self.down_conv_2 = down_conv(64,128,4,2,1)
        self.down_conv_3 = down_conv(128,256,4,2,1)
        self.down_conv_4 = down_conv(256,512,4,2,1)
        self.down_conv_5 = down_conv(512,1024,4,2,1)

        self.up_trans_1 = up_conv(1024, 512, 4, 2, 1)
        self.up_conv_1 = down_conv(1024,512, 3, 1, 1)
        self.up_trans_2 = up_conv(512, 256, 4, 2, 1)
        self.up_conv_2 = down_conv(512, 256, 3, 1, 1)
        self.up_trans_3 = up_conv(256, 128, 4, 2, 1)
        self.up_conv_3 = down_conv(256, 128, 3, 1, 1)
        self.up_trans_4 = up_conv(128, 64, 4, 2, 1)
        self.up_conv_4 = down_conv(128, 64, 3, 1, 1)

        self.out = nn.Conv2d(64, 3, kernel_size=1, stride=1, padding=0)

    def forward(self, image):
      #encoder
                                   # image = [32, 3, 256, 256]
      x1 = self.down_conv_1(image) # x1 = [32, 64, 256, 256]
      x2 = self.down_conv_2(x1)    # x2 = [32, 128, 128, 128]
      x3 = self.down_conv_3(x2)    # x3 = [32, 256, 64, 64]
      x4 = self.down_conv_4(x3)    # x4 = [32, 512, 32, 32]
      x5 = self.down_conv_5(x4)    # x5 = [32, 1024, 16, 16]

      #decoder
      y = self.up_trans_1(x5)                   # y1 = [32, 512, 32, 32]
      y = self.up_conv_1(torch.cat([y,x4],1))  # y1 = [32, 512, 32, 32]
      y = self.up_trans_2(y)                   # y2 = [32, 256, 64, 64]
      y = self.up_conv_2(torch.cat([y,x3],1))  # y2 = [32, 256, 64, 64]
      y = self.up_trans_3(y)                   # y3 = [32, 128, 128, 128]
      y = self.up_conv_3(torch.cat([y,x2],1))  # y3 = [32, 128, 128, 128]
      y = self.up_trans_4(y)                   # y4 = [32, 64, 256, 256]
      y = self.up_conv_4(torch.cat([y,x1],1))  # y4 = [32, 64, 256, 256]
      y = self.out(y)

      return y

In [ ]:
# FUNCTIONS TO CREATE DATALOADERS

def custom_transform(x):
    return x / torch.max(x.abs())



def create_dataloader(directory, batch_size=32, mean=0.5, std=0.5):
    # transform = transforms.Compose([
    #     transforms.ToTensor(),   
    #     transforms.Normalize(mean=mean, std=std)
    # ])
    
    transform = transforms.Compose([
        transforms.Resize((256, 256)),  # Resize images
        transforms.ToTensor(),           # Convert to tensor
        custom_transform                  # Using the custom transform function
    ])

    dataset = DehazingDataset(directory, transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    return dataloader

def create_train_val_dataloaders(root_dir, train_batch_size=32, val_batch_size=32, mean=0.5, std=0.5):
    train_dir = os.path.join(root_dir, 'train')
    val_dir = os.path.join(root_dir, 'val')

    train_dataloader = create_dataloader(train_dir, batch_size=train_batch_size, mean=mean, std=std)
    val_dataloader = create_dataloader(val_dir, batch_size=val_batch_size, mean=mean, std=std)

    return train_dataloader, val_dataloader

In [ ]:
# WHERE VAL AND TRAIN FOLDERS ARE PRESENT
root_dir = '../Task2Dataset'
train_dataloader, val_dataloader = create_train_val_dataloaders(root_dir)

In [ ]:
std1 = 0.5
std2 = np.array([0.229, 0.224, 0.225])

mean1 = 0.5
mean2 = np.array([0.485, 0.456, 0.406])

In [ ]:
def show_images(hazy_imgs, clean_imgs, generated_imgs, mean, std, num_images=5, save_path = None):
    fig, axes = plt.subplots(3, num_images, figsize=(15, 10))
    for i in range(num_images):
        clean_image = clean_imgs[i].detach().permute(1, 2, 0).cpu().numpy()
        hazy_image = hazy_imgs[i].detach().permute(1, 2, 0).cpu().numpy()
        generated_image = generated_imgs[i].detach().permute(1, 2, 0).cpu().numpy()
    

        clean_image = clean_image * mean + std
        hazy_image = hazy_image * mean + std
        # generated_image = generated_image * mean + std

        
        # Plot hazy images
        axes[0, i].imshow(hazy_image)
        axes[0, i].axis('off')
        axes[0, i].set_title("Hazy Image")

        # Plot clean images
        axes[1, i].imshow(clean_image)
        axes[1, i].axis('off')
        axes[1, i].set_title("Clean Image")

        # Plot generated images
        axes[2, i].imshow(generated_image)
        axes[2, i].axis('off')
        axes[2, i].set_title("Generated Image")

    plt.tight_layout()

    if save_path:
        plt.savefig(save_path)
    plt.show()

In [ ]:
# INITIALISE AND LOAD WEIGHTS TO A GENERATOR

def load_generator_weights(generator, weights_path):
    map_location = torch.device('cpu')
    generator.load_state_dict(torch.load(weights_path, map_location=map_location))
    generator.eval()

    return generator
    

generator = GeneratorModel4()

weights_path = '../Models/generator_epoch_4.pth'

generator = load_generator_weights(generator, weights_path)

In [ ]:
def visualise_images_from_dataloader(val_dataloader, total_images = 40):
    count = 0
    for hazy_imgs, clean_imgs in val_dataloader:
        generated_imgs = generator(hazy_imgs)
        show_images(hazy_imgs, clean_imgs, generated_imgs, num_images = 5 )
        count += 5

        if count >= total_images:
            break

In [ ]:
# FUNCTIONS TO GET MEAN/MEDIAN PSNR SCORES AND SIMILARITY
def PSNR(img1, img2):
    """Calculate the Peak Signal-to-Noise Ratio (PSNR) between two images."""
    mse = np.mean((img1 - img2) ** 2)
    if mse == 0:  
        return 100
    max_pixel = 255.0
    psnr = 10 * log10(max_pixel**2 / mse)
    return psnr

def compute_PSNR(imgs1, imgs2):
    """Calculate PSNR for a batch of images."""
    psnr_scores = []
    for img1, img2 in zip(imgs1, imgs2):
        img1_np = img1.detach().cpu().numpy()
        img2_np = img2.detach().cpu().numpy()
        psnr_scores.append(PSNR(img1_np, img2_np))
    return psnr_scores

def compute_SSIM(imgs1, imgs2):
    """Calculate SSIM for a batch of images."""
    ssim_scores = []
    for img1, img2 in zip(imgs1, imgs2):
        img1_np = img1.detach().cpu().numpy()
        img2_np = img2.detach().cpu().numpy()
        
        # Resize images if they are too small
        min_size = min(img1_np.shape[0], img1_np.shape[1])
        if min_size < 7:
            ratio = 7 / min_size
            img1_np = resize(img1_np, (int(img1_np.shape[0] * ratio), int(img1_np.shape[1] * ratio)))
            img2_np = resize(img2_np, (int(img2_np.shape[0] * ratio), int(img2_np.shape[1] * ratio)))
        
        ssim_scores.append(ssim(img1_np, img2_np, multichannel=True, data_range=1))
    return ssim_scores


def get_mean_score(scores):
    """Calculate the mean score from a list of scores."""
    mean_score = np.mean(scores)
    return mean_score

def get_median_score(scores):
    """Calculate the median score from a list of scores."""
    median_score = np.median(scores)
    return median_score

def denormalize(img, mean=0.5, std=0.5):
    """Denormalize a single image."""
    mean = np.array(mean)
    std = np.array(std)
    denormalized_img = img * std + mean
    return denormalized_img

def print_scores(val_dataloader, generator, mean=0.5, std=0.5):
    psnr_scores = []
    ssim_scores = []
    i = 1
    for hazy_imgs, clean_imgs in val_dataloader:
        with torch.no_grad():
            generated_imgs = generator(hazy_imgs)
        # generated_imgs = denormalize(generated_imgs, mean, std)
        # clean_imgs = denormalize(clean_imgs, mean, std)
        generated_imgs = generated_imgs * 255
        clean_imgs = clean_imgs * 255
        psnr_batch = compute_PSNR(clean_imgs, generated_imgs)
        ssim_batch = compute_SSIM(clean_imgs, generated_imgs)
        psnr_scores.extend(psnr_batch)
        ssim_scores.extend(ssim_batch)

        print(f"{i}th Batch Done")
        i += 1

    mean_psnr = get_mean_score(psnr_scores)
    median_psnr = get_median_score(psnr_scores)
    mean_ssim = get_mean_score(ssim_scores)
    median_ssim = get_median_score(ssim_scores)

    print(f"Mean PSNR: {mean_psnr:.2f}, Median PSNR: {median_psnr:.2f}")
    print(f"Mean SSIM: {mean_ssim:.4f}, Median SSIM: {median_ssim:.4f}")

    return psnr_scores, ssim_scores

In [ ]:
psnr_scores, ssim_scores = print_scores(val_dataloader, generator)

In [ ]:
np.max(psnr_scores), np.min(psnr_scores)

In [ ]:
np.max(ssim_scores), np.min(ssim_scores)